In [17]:
from grover.data import MoleculeDatapoint, MoleculeDataset, StandardScaler
import csv
from tqdm import tqdm
from torch.utils.data import DataLoader
from grover.data import MolCollator 
from argparse import ArgumentParser, Namespace


In [18]:
args = Namespace()
args.no_cache = True
args.bond_drop_rate = 0


In [19]:
path = "/home/pj20/gode/data_process/valid_smiles.csv"

In [20]:
with open(path) as f:
    reader = csv.reader(f)
    next(reader)  # skip header

    lines = []
    for line in reader:
        smiles = line[0]
        lines.append(line)

    data = MoleculeDataset([
        MoleculeDatapoint(
            line=line,
        ) for i, line in tqdm(enumerate(lines), total=len(lines), disable=True)
    ])

In [22]:
mol_collator = MolCollator(shared_dict={}, args=args)
train_data = DataLoader(data,
                        batch_size=64,
                        num_workers=10,
                        collate_fn=mol_collator)

In [25]:
for item in train_data:
    print(item)
    break

(['CC1COC2=C3N1C=C(C(=O)C3=CC(=C2N4CCN(CC4)C)F)C(=O)O', '[H]Oc1nn([H])c(C([H])([H])[H])c1C([H])([H])[H]', 'C=C(C)C(=O)OCCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F', 'C1CCCNCCC1', 'CNC(=O)C[C@@H](N)C(=O)N[C@@H](C(=O)N[C@@H]1C(=O)N2[C@@H](C(=O)O)C(C)(C)S[C@H]12)c1ccc(O)cc1', '[H]C1=C([H])C([H])(C([H])([H])C([H])([H])[H])C([H])([H])C1([H])C([H])([H])[H]', 'N=C1NC(=N)c2ccccc21', '[H]OC([H])([H])c1c([H])oc([H])c1C([H])([H])[H]', 'C1=CC=CC=C1CN2C(CC(C2)CN)=O', 'OC1=CC(Cl)=C(Cl)C(Cl)=C1', '[H]OC1([H])C([H])([H])C([H])=C([H])C([H])([H])C1([H])C([H])([H])[H]', '[H]C1=NC([H])([H])C([H])([H])C(=O)O1', 'C[C@H]1[C@H]2[C@H](C[C@H]3[C@@H]4CC[C@H]5C[C@@H](O)CC[C@]5(C)[C@H]4CC(=O)[C@]23C)O[C@]11CC[C@@H](C)CO1', 'O(C(C)(C)C)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](Cc1ccccc1)C(=O)N[C@H]([C@@H](O)C[C@H](C(=O)NCC(C)C)C)CC(C)C', 'CC[C@@H](N)CO', 'COCC(=O)N(C(C)C(=O)OC)C1=C(C)C=CC=C1C', '[H]C(=O)C1(C#N)C([H])([H])C1([H])[H]', 'CCCCCCCCCCCCCCCC', 'CC(=O)O[C@H]1CC[C@@]2(C)C(=CC[C@H]3[C@@H]4CC=C(C(C)=O)[C@@]4(C)CC[C@

In [1]:
import torch

path = "/data/pj20/grover/pretrain/grover_large.pt"

state = torch.load(path, map_location=lambda storage, loc: storage)

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
args, loaded_state_dict = state['args'], state['state_dict']

In [13]:
args

Namespace(activation='PReLU', backbone='dualtrans', bias=False, bond_drop_rate=0, cuda=True, dense=False, depth=6, dist_coff=0.1, dropout=0.0, embedding_output_type='both', fine_tune_coff=1, hidden_size=1200, input_layer='fc', no_attach_fea=True, num_attn_head=4, num_mt_block=1, select_by_loss=True, self_attention=False, skip_epoch=0, undirected=False, weight_decay=2e-07)

In [3]:
from mgnn import build_model

m_gnn = build_model(args=args)

In [5]:
model_state_dict = m_gnn.state_dict()

In [ ]:
pretrained_state_dict = {}
for param_name in loaded_state_dict.keys():
    new_param_name = param_name
    if new_param_name not in model_state_dict:
        print(f'Pretrained parameter "{param_name}" cannot be found in model parameters.')
    elif model_state_dict[new_param_name].shape != loaded_state_dict[param_name].shape:
        print(f'Pretrained parameter "{param_name}" '
                f'of shape {loaded_state_dict[param_name].shape} does not match corresponding '
                f'model parameter of shape {model_state_dict[new_param_name].shape}.')
    else:
        print(f'Loading pretrained parameter "{param_name}".')
        pretrained_state_dict[new_param_name] = loaded_state_dict[param_name]

model_state_dict.update(pretrained_state_dict)
m_gnn.load_state_dict(model_state_dict)